# 0.1. IMPORTS

In [1]:
import pandas as pd
from statsmodels.stats import api as sms
import math
from scipy.stats import ttest_1samp, shapiro, ttest_ind, mannwhitneyu, f_oneway
import numpy as np
import seaborn as sns
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import chi2_contingency

# 1.0. LOADING DATA

In [2]:
df = pd.read_csv(r"C:\Users\Admin\Downloads\projeto_teste_AB\dataset\ab_data.csv") # r serve para abrir a pasta no windows
df

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1
...,...,...,...,...,...
294473,751197,2017-01-03 22:28:38.630509,control,old_page,0
294474,945152,2017-01-12 00:51:57.078372,control,old_page,0
294475,734608,2017-01-22 11:45:03.439544,control,old_page,0
294476,697314,2017-01-15 01:20:28.957438,control,old_page,0


# 2.0. DESIGN DE EXPERIMENTS

## 2.1. PARAMETER OF EXPERIMENTS

In [3]:
# Nível de Confiança
confidence_level = 0.95

# Nível de Significância
significance_level = (1 - confidence_level)

# Conversões da página atual e nova
p1 = 0.13
p2 = 0.15

# Tamanho do efeito
effect_size = sms.proportion_effectsize(p1, p2)

# Poder Estatístico
power = 0.80

# Tamanho da amostra
sample_n = sms.NormalIndPower().solve_power(
    effect_size,
    power = power,
    alpha = significance_level
)

sample_n = math.ceil(sample_n) # tira as casas decimais depois do ponto
sample_n

4720

In [4]:
# Preparar os dados
df_aux = df[["user_id", "group"]].groupby("user_id").count().reset_index().query("group > 1")
df3 = df[~df['user_id'].isin(df_aux['user_id'])] # o ~ significa que é negativo está removendo os usuários e o isin filtra dataframe

# Amostragem
df_control_sample = df3[df3["group"] == "control"].sample(n = sample_n, random_state=32)
print(f'Size of Control group: {df_control_sample.shape[0]}') # shape é para mostrar a quantidade de linhas e colunas

df_treatment_sample = df3[df3["group"] == "treatment"].sample(n = sample_n, random_state=32)
print(f'Size of Control group: {df_treatment_sample.shape[0]}') # shape é para mostrar a quantidade de linhas e colunas

df_ab = pd.concat([df_control_sample, df_treatment_sample])

# Taxa de Conversão
converted = df_control_sample.loc[df_control_sample["converted"] == 1, "converted"].sum()
conversion_rate_control = converted / len(df_control_sample)
print(f'\nConversion Rate - Control Group: {conversion_rate_control:.6f}')

converted = df_treatment_sample.loc[df_treatment_sample['converted'] == 1, 'converted'].sum()
conversion_rate_treatment = converted / len(df_treatment_sample)
print(f'Conversion Rate - Treatment Group: {conversion_rate_treatment:.6f}')

# Teste de Hipóteses
df_table = df_ab[["group", "converted"]].groupby("group").agg({'converted':['sum', 'count']}) # nesse caso o nome da coluna é "converted" e o agg cria duas colunas "sum" "count"
df_table.columns = ['converted', 'non_converted'] # está renomeando as colunas

df_table = df_ab[['group', 'converted']].groupby('group').agg({'converted' :['sum', 'count']})
df_table.columns = ['converted', 'non_converted']

chi_val, pval, dof, expected = chi2_contingency(df_table)

print(f'p-value: {pval:.2f}')

if pval < significance_level:
    print('Rejeita a hipóteses nula')
else:
    print('Falha em rejeitar a hipótese nula')

Size of Control group: 4720
Size of Control group: 4720

Conversion Rate - Control Group: 0.118644
Conversion Rate - Treatment Group: 0.119703
p-value: 0.91
Falha em rejeitar a hipótese nula


In [5]:
df_control_sample.shape

(4720, 5)

# 3.0. Conversão da página em faturamento

In [6]:
#conversão de resultado para R$
pagina_atual = 13 / 100
pagina_nova = 15 / 100

#Media de visitantes diários * taxa de conversão da pagina atual = compradores
#Compradores * tick medio = faturamento (GMV - Gross margin Value)

In [ ]:
df4 = df3.copy()

df4['timestamp'] = pd.to_datetime(df4['timestamp']).apply(lambda x: x.strftime('%Y-%m-%d')) # está convertendo para datetime

In [ ]:
df5 = df4[["user_id", "timestamp"]].groupby("timestamp").count().reset_index()

# Current GMV gross Margin value
df5['current_purchases'] = np.ceil(df5["user_id"] * pagina_atual).astype(int) # ceil está arredondando, esse é a página atual

df5['current_GMV'] = df5['current_purchases'] * 4500

current_gmv = df5['current_GMV'].sum()
print(f'GMV On Period: {current_gmv}')

# Expected GMV
df5['new_purchases'] = np.ceil(df5['user_id'] * pagina_nova).astype(int) # esse é a página nova
df5['new_GMV'] = df5['new_purchases'] * 4500

new_gmv = df5['new_GMV'].sum()
print(f'New GMV On Period: {new_gmv}')

lift_abs = new_gmv - current_gmv # diferença entre a nova página e a página atual
print(f'Abs lift: {lift_abs}')

lift = 100 * (new_gmv - current_gmv) / current_gmv # percentual de ganho a mais para a nova página
print(f'Expected Lift: {lift:.2f}%') 


GMV On Period: 167760000
New GMV On Period: 193563000
Abs lift: 25803000
Expected Lift: 15.38%
